In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.array as da
import os
import sys
import logging


logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.Analysis import Analysis, AnalysisManager, GR_Confirmation, SpotDetection_Confirmation

c:\Users\Jack\Documents\GitHub\AngelFISH


Loads in the data from specified location

In [3]:
loc = r"Y:\smFISH_images\Eric_smFISH_images\20230511\GR_ICC_3hr_R1_1nM_0min_050823\GR_ICC_3hr_R1_1nM_0min_050823.h5"
log_location = None # r'Y:\Users\Jack\All_Analysis'
am = AnalysisManager(location=loc, log_location=log_location) # change this to your mounted drive location for this folder
# This analysis is best done with the NAS mounted 

PermissionError: [Errno 13] Unable to synchronously open file (unable to open file: name = 'Y:\smFISH_images\Eric_smFISH_images\20230511\GR_ICC_3hr_R1_1nM_0min_050823\GR_ICC_3hr_R1_1nM_0min_050823.h5', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)

In [ ]:
# list all analysis done 
am.list_analysis_names()

In [ ]:
# can filter on name and dates
am.select_analysis('ER_Dec0324_GR')
print(am.analysis)

Does analysis/confirmation

In [5]:
# select GR conformation analysis 
GR = GR_Confirmation(am)

In [6]:
# this loads the data into memory 
GR.get_data()

In [ ]:
# This is a temporary fix, I didnt do this in the correct spot in the first run
# this will be fixed in future runs
for h5 in np.unique(GR.cellprops['h5_idx']):
    GR.cellprops.loc[GR.cellprops['h5_idx'] == h5, 'fov'] = pd.Categorical(GR.cellprops.loc[GR.cellprops['h5_idx'] == h5, 'fov']).codes


In [ ]:
# make sure this has all the measurements you want
GR.cellprops.keys()

In [ ]:
# run this multiple times to see a new randomly selected cell
GR.display()

In [ ]:
GR.cellprops

In [ ]:
keys_to_plot = ['area', 'intensity_mean']
channel_maps = {'-1': 'nuc', '-0': 'GR'}
for k in GR.cellprops.keys():
    if keys_to_plot in keys_to_plot:
        # Plot histogram for 'area'
        plt.figure(figsize=(10, 5))
        plt.hist(GR.cellprops[k], bins=30, alpha=0.5, label='Area')
        plt.ylabel('Frequency')
        for suffix, channel in channel_maps.items():
            if k.endswith(suffix):
                k = k.replace(suffix, channel)
                break
        plt.title(f'Histogram of {k}')
        plt.legend()
        plt.show()

In [ ]:
GR.close()